In [0]:
spark.sql('CREATE SCHEMA IF NOT EXISTS workspace.iris_gold;')

In [0]:
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, TimestampType

iris_gold_schema = StructType([
    StructField('sepal_length', DoubleType(), True),
    StructField('sepal_width', DoubleType(), True),
    StructField('petal_length', DoubleType(), True),
    StructField('petal_width', DoubleType(), True),
    StructField('species', StringType(), True),
    StructField('sepal_area', DoubleType(), True),
    StructField('petal_area', DoubleType(), True),
    StructField('petal_ratio', DoubleType(), True),
])


In [0]:
from pyspark.sql import functions as F

df_silver = spark.table('workspace.iris_silver.iris_silver')

df_gold = (df_silver
    .withColumn("sepal_area", F.col("sepal_length") * F.col("sepal_width"))
    .withColumn("petal_area", F.col("petal_length") * F.col("petal_width"))
    .withColumn("petal_ratio", F.col("petal_length") / F.col("petal_width"))
)

df_gold = df_gold.select(
    F.col("sepal_length").cast("double").alias("sepal_length"),
    F.col("sepal_width").cast("double").alias("sepal_width"),
    F.col("petal_length").cast("double").alias("petal_length"),
    F.col("petal_width").cast("double").alias("petal_width"),
    F.col("species").cast("string").alias("species"),
    F.col("sepal_area").cast("double").alias("sepal_area"),
    F.col("petal_area").cast("double").alias("petal_area"),
    F.col("petal_ratio").cast("double").alias("petal_ratio"),
)

df_gold.write.mode("overwrite").format("delta").saveAsTable("workspace.iris_gold.df_iris_gold")